In [ ]:
from dotenv import load_dotenv, find_dotenv
from pathlib import Path
from tqdm import tqdm
import sys

_ = load_dotenv(find_dotenv())  # read local .env file
sys.path.append(Path("..").resolve().as_posix())

In [ ]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

from src.core.index import Index, ID_KEY
from src.core.retriever import build_retriever
from src.core.generator import build_generator

In [ ]:
persist_path = Path("index_storage").resolve()
embedding_function = AzureOpenAIEmbeddings(
    azure_deployment="text-embedding-ada-002", api_version="2023-07-01-preview"
)

if persist_path.exists():
    index = Index.from_persist_path(
        persist_path=persist_path,
        embedding_function=embedding_function,
    )
else:
    index = Index.from_metadata(
        metadata_path=Path("../resources/doc_metadata.jsonl").resolve(),
        lib_path=Path("../../docs_md").resolve(),
        persist_path=persist_path,
        embedding_function=embedding_function,
    )

In [ ]:
llm = AzureChatOpenAI(
    temperature=0.0,
    azure_deployment="gpt4o",
    openai_api_version="2023-07-01-preview",
    max_retries=0,
)

In [ ]:
retriever = build_retriever(
    llm=llm, vectorstore=index.vectorstore, docstore=index.docstore
)

In [ ]:
generator = build_generator(llm=llm, retriever=retriever)

In [ ]:
from langchain.globals import set_debug

set_debug(False)

response = generator.invoke(
    {"input": "Can I use row-level security on EdgeDB?"},
    config={
        "configurable": {"session_id": "1"}
    },  # constructs a key "abc123" in `store`.
)

In [ ]:
def pretty_print_response(response):
    print(f"************PROMPT************\n\n{response['input']}\n\n")
    print(f"************ANSWER************\n\n{response['answer'].answer}\n\n")

    print(f"***********CITATIONS**********\n\n")

    for citation in response["answer"].citations:
        doc = response["retrieval_result"]["documents"][citation.source_id]
        print(f"Source {citation.source_id}: {doc.metadata['source']}\n")
        print(f"Category: {doc.metadata['category']}\n")
        print(f"Quote:\n{citation.quote}\n\n")

    print(f"*********SEARCH TERMS*********\n")
    search_terms = response["retrieval_result"]["search_terms"]
    print(f"Query: {search_terms.query}")
    print(f"Filter: {search_terms.category}\n\n")

    print(f"************SOURCES***********\n\n")

    for i, doc in enumerate(response["retrieval_result"]["documents"]):
        print(f"Source {i}: {doc.metadata['source']}\n")
        print(f"Category: {doc.metadata['category']}\n")
        print(f"Content:\n{doc.page_content}\n\n")

In [ ]:
pretty_print_response(response)

In [ ]:
response